In [100]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.common.keys import Keys
import subprocess
import json
import time as time

In [103]:
driver = webdriver.Chrome('../chromedriver_linux64/chromedriver')

In [104]:
driver.get('https://www.epa.gov/outdoor-air-quality-data/download-daily-data')

In [97]:
f = open('polltant_names.json')
print(json.load(f))

['CO', 'Pb', 'NO2', 'Ozone', 'PM10', 'PM2.5', 'SO2']


In [98]:
f = open('state_names.json')
print(json.load(f))

['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Puerto Rico']


In [101]:
selected_pollutant='CO'
selected_state='Illinois'

In [102]:
driver.refresh()

select = Select(driver.find_element(By.NAME,'poll'))
pollutants=select.options

pollutant_names=[]
for p in pollutants:
    pollutant_names.append(p.text)

p_index=pollutant_names.index(selected_pollutant)

for p in range(p_index,p_index+1):

    select = Select(driver.find_element(By.NAME,'poll'))
    select.select_by_index(p)
    pollutant=select.first_selected_option.text
    time.sleep(1)

    select = Select(driver.find_element(By.NAME,'year'))
    years=select.options

    print(f'Data available from {years[2].text} to {years[-1].text}')

    for y in range(2,len(years)):

        select = Select(driver.find_element(By.NAME,'year'))
        select.select_by_index(y)
        year=select.first_selected_option.text
        time.sleep(1)

        select = Select(driver.find_element(By.NAME,'state'))
        states=select.options

        state_names=[]
        for s in states:
            state_names.append(s.text)

        s_index=state_names.index(selected_state)

        for s in range(s_index,s_index+1):

            select = Select(driver.find_element(By.NAME,'state'))
            select.select_by_index(s)
            state=select.first_selected_option.text
            time.sleep(1)

            driver.find_element(By.XPATH,'//*[@id="launch"]/input').click()
            time.sleep(10)
            driver.find_element(By.PARTIAL_LINK_TEXT,'CSV').click()
            time.sleep(10)

            name=str(pollutant)+'-'+str(year)+'-'+str(state)
            subprocess.run(['mv ../ad_viz_plotval_data.csv DataAQ/{}'.format(name)],shell=True) 
            print(f'{pollutant}-{year}-{state}')

Data available from 2022 to 1980
CO-2022-Illinois
CO-2021-Illinois
CO-2020-Illinois
CO-2019-Illinois
CO-2018-Illinois
CO-2017-Illinois
CO-2016-Illinois
CO-2015-Illinois
CO-2014-Illinois
CO-2013-Illinois
CO-2012-Illinois
CO-2011-Illinois
CO-2010-Illinois
CO-2009-Illinois
CO-2008-Illinois
CO-2007-Illinois
CO-2006-Illinois
CO-2005-Illinois
CO-2004-Illinois
CO-2003-Illinois
CO-2002-Illinois
CO-2001-Illinois
CO-2000-Illinois
CO-1999-Illinois
CO-1998-Illinois
CO-1997-Illinois
CO-1996-Illinois
CO-1995-Illinois
CO-1994-Illinois
CO-1993-Illinois
CO-1992-Illinois
CO-1991-Illinois
CO-1990-Illinois
CO-1989-Illinois
CO-1988-Illinois
CO-1987-Illinois
CO-1986-Illinois
CO-1985-Illinois
CO-1984-Illinois
CO-1983-Illinois
CO-1982-Illinois
CO-1981-Illinois
CO-1980-Illinois
